In [1]:
import geopandas as gpd
import time


In [4]:
gdf = gpd.read_file("./example-data/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp")

In [5]:
inside_symbol="#"
outside_symbol=" "
width=100
crs=4326

# dissolve data
gdf = gdf.to_crs(crs)
gdf["is_gdf"] = True
gdf['char'] = 'w'
gdf = gdf.dissolve(by="is_gdf")


    
# FORMAT ASCII EXTENT
# get geometrical extents
minx, miny, maxx, maxy = gdf.total_bounds
x_extent = maxx - minx
y_extent = maxy - miny

# define ascii max size
maxchars = width
# space between rows = about 2x character space
row_space = 2

# account for higher rowspace
ratio = (x_extent / y_extent) * row_space
# line length
ncols = maxchars
# number of rows
nrows = int(maxchars / ratio)

## Uncomment to automatically scale ascii depending on horizontal and vertical orientation
#if x_extent > y_extent:
#    # account for higher rowspace
#    ratio = (x_extent / y_extent) * row_space
#    # line length
#    ncols = maxchars
#    # number of rows
#    nrows = int(maxchars / ratio)
#else:
#    # account for higher rowspace
#    maxchars = maxchars / row_space
#    ratio = (y_extent / x_extent) / row_space
#    # line length
#    ncols = int(maxchars / ratio)
#    # number of rows
#    nrows = int(maxchars)

    
# GENERATE A GEOMETRICAL POINT FOR EACH ASCII CHARACTER
# axes as iterable lists
y_axis_i = list(range(nrows))
x_axis_i = list(range(ncols))

# x and y Distance between points
x_step = x_extent/ncols
y_step = y_extent/nrows

# generate points
points = gpd.GeoDataFrame()

count = 0

# loop rows
for iy in y_axis_i:
        
    # y coord
    y_coord = maxy - y_step * iy

    # loop cols
    for ix in x_axis_i:
        x_coord = minx + x_step * ix

        # add to gdf
        points.at[count, "x_coord"] = x_coord
        points.at[count, "y_coord"] = y_coord

        # gdf index
        count = count + 1

# create geometries
points["geometry"] = gpd.points_from_xy(points["x_coord"], points["y_coord"])
points = points.set_crs(epsg=crs)

In [13]:
points = points.sjoin(gdf, how='left')
points['char'] = points['char'].fillna(' ')
points

,x_coord,y_coord,geometry,index_right,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,...,FC_TR,FC_ID,FC_PL,FC_GR,FC_IT,FC_NL,FC_SE,FC_BD,FC_UA,char
0,-180.0,83.599609,POINT (-180.00000 83.59961),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,-176.4,83.599609,POINT (-176.40000 83.59961),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,-172.8,83.599609,POINT (-172.80000 83.59961),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,-169.2,83.599609,POINT (-169.20000 83.59961),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,-165.6,83.599609,POINT (-165.60000 83.59961),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,162.0,-82.765653,POINT (162.00000 -82.76565),True,Admin-0 country,1.0,3.0,Zimbabwe,ZWE,0.0,...,Unrecognized,Unrecognized,Unrecognized,Unrecognized,Unrecognized,Unrecognized,Unrecognized,Unrecognized,Unrecognized,w
2396,165.6,-82.765653,POINT (165.60000 -82.76565),True,Admin-0 country,1.0,3.0,Zimbabwe,ZWE,0.0,...,Unrecognized,Unrecognized,Unrecognized,Unrecognized,Unrecognized,Unrecognized,Unrecognized,Unrecognized,Unrecognized,w
2397,169.2,-82.765653,POINT (169.20000 -82.76565),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2398,172.8,-82.765653,POINT (172.80000 -82.76565),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [9]:
points

,x_coord,y_coord,geometry
0,-180.0,83.599609,POINT (-180.00000 83.59961)
1,-176.4,83.599609,POINT (-176.40000 83.59961)
2,-172.8,83.599609,POINT (-172.80000 83.59961)
3,-169.2,83.599609,POINT (-169.20000 83.59961)
4,-165.6,83.599609,POINT (-165.60000 83.59961)
...,...,...,...
2395,162.0,-82.765653,POINT (162.00000 -82.76565)
2396,165.6,-82.765653,POINT (165.60000 -82.76565)
2397,169.2,-82.765653,POINT (169.20000 -82.76565)
2398,172.8,-82.765653,POINT (172.80000 -82.76565)


In [ ]:



# ASSIGN ASCII CHARACTER TO EACH POINT
# see if each point is within the gdf

within = points.overlay(gdf, how='intersection')
within["char"] = inside_symbol
not_within = points.overlay(gdf, how='difference')
not_within["char"] = outside_symbol
    
# merge data
points = within.append(not_within)

    
# FORMAT DATA FROM GEOMETRIES TO ASCII
# rows as groups
row_grouped = points.sort_values('x_coord').groupby("y_coord")

# list for rows
rows = []
    
# Format row strings
for key, group in row_grouped:
    row_list = group["char"].tolist()
    row_str = "".join(map(str, row_list))
    rows.append(row_str)
        
# print ascii output
for r in reversed(rows):
    print(r)
    time.sleep(0.2)



#world = gpd.read_file("./example-data/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp")
#ascify(gdf=world, crs=4326, inside_symbol="#", outside_symbol=" ")






